#### Install the useful packages.

In [4]:
myPackages <- c("easypackages")
#the argument `lib.loc = "~/rlib"` is used because the packages were installed in this folder in my case
lapply(myPackages, library, lib.loc = "~/rlib", character.only = TRUE) 
libraries("dplyr", "tidyr")

[[1]]
[1] "easypackages" "stats"        "graphics"     "grDevices"    "utils"       
[6] "datasets"     "methods"      "base"

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: tidyr

All packages loaded successfully



#### Import the gallup data that contain every country.

In [8]:
gallupRaw <- rio::import("~/gallup data/The_Gallup_042722.dat")

Ambiguous file format ('.dat'), but attempting 'data.table::fread("~/gallup data/The_Gallup_042722.dat")'



We need country name, country code, weight, survey date, survey year, ls, hope, PA, NA

Filter the data and save the it for future use.

In [9]:
gallupExtracted <- gallupRaw %>% select(COUNTRYNEW, COUNTRY_ISO3, WGT, FIELD_DATE, 
                                        YEAR_WAVE, YEAR_CALENDAR, ls = WP16, 
                                        hope = WP18, smile = WP63, enjoy = WP67, 
                                        worry = WP69, sad = WP70, stress = WP71,
                                        anger = WP74)
write.csv(gallupExtracted, "gallupExtracted.csv", row.names = FALSE)

In [1]:
gallupExtracted <- rio::import("gallupExtracted.csv")
colnames(gallupExtracted)
dim(gallupExtracted)

[1] "COUNTRYNEW"    "COUNTRY_ISO3"  "WGT"           "FIELD_DATE"   
 [5] "YEAR_WAVE"     "YEAR_CALENDAR" "ls"            "hope"         
 [9] "smile"         "enjoy"         "worry"         "sad"          
[13] "stress"        "anger"

[1] 2450403      14

Create a function for changing the "DK" / "Refused" response to NA.

In [3]:
na_codes <- function(x, ...) {
    for(y in list(...))
        x[x == y] <- NA
    x
}

Change the "DK" / "Refused" response to NA

Change the responses to SWB from [1 Yes, 2 No] to [1 Yes, 0 No] for getting the means for individual positive affect and negative affect

In [5]:
gallupAssas <- gallupExtracted
#life satisfication
gallupAssas$ls <- na_codes(gallupAssas$ls, 98, 99)
#hope
gallupAssas$hope <- na_codes(gallupAssas$hope, 98, 99)
#pa and #na
gallupAssas$smile <- na_codes(gallupAssas$smile, 3, 4)
gallupAssas$enjoy <- na_codes(gallupAssas$enjoy, 3, 4)
gallupAssas$worry <- na_codes(gallupAssas$worry, 3, 4)
gallupAssas$sad <- na_codes(gallupAssas$sad, 3, 4)
gallupAssas$stress <- na_codes(gallupAssas$stress, 3, 4)
gallupAssas$anger <- na_codes(gallupAssas$anger, 3, 4)
gallupAssas <- gallupAssas %>% 
 rowwise() %>% 
 mutate(pa = mean(cbind(enjoy, smile), na.rm=TRUE),
        na = mean(cbind(worry, sad, stress, anger), na.rm=TRUE)) %>% ungroup
head(gallupAssas)

COUNTRYNEW,COUNTRY_ISO3,WGT,FIELD_DATE,YEAR_WAVE,YEAR_CALENDAR,ls,hope,smile,enjoy,worry,sad,stress,anger,pa,na
<chr>,<chr>,<dbl>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
United States,USA,0.6689163,7/1/2006,2006,2006,6,8,1,1,2,2,2,2,1,2.0
United States,USA,0.1966555,7/1/2006,2006,2006,8,NA,1,1,2,2,2,2,1,2.0
United States,USA,1.3032635,7/1/2006,2006,2006,8,8,1,1,1,2,1,2,1,1.5
United States,USA,0.6359364,7/1/2006,2006,2006,8,8,1,1,2,2,2,2,1,2.0
United States,USA,0.7358092,7/1/2006,2006,2006,10,10,1,1,2,2,2,2,1,2.0
United States,USA,1.7369582,7/1/2006,2006,2006,5,10,1,1,1,1,1,1,1,1.0


In [6]:
write.csv(gallupAssas, "gallupAssas.csv")

In [7]:
colnames(gallupAssas)
dim(gallupAssas)

[1] "COUNTRYNEW"    "COUNTRY_ISO3"  "WGT"           "FIELD_DATE"   
 [5] "YEAR_WAVE"     "YEAR_CALENDAR" "ls"            "hope"         
 [9] "smile"         "enjoy"         "worry"         "sad"          
[13] "stress"        "anger"         "pa"            "na"

[1] 2450403      16